In [10]:
using DataFrames, Distributions, DataFramesMeta

In [11]:
function gen_data(
    ; 
    n = 500000, 
    mu = [0, 2], 
    sigma = 1, 
    pZ1 = 0.8)
    
    data = DataFrame(
        Obs_ID = 1:n,
        Z = (rand(Uniform(0,1), n) .> pZ1) .+ 1
        )

    @transform!(data, :Y = rand(Normal(0, sigma), n) + mu[:Z])    
    @select!(data, :Obs_ID, :Y)
    @transform!(data, :p_Y_given_Z_1 = pdf.(Normal(mu[1], sigma), :Y))
    @transform!(data, :p_Y_given_Z_2 = pdf.(Normal(mu[2], sigma), :Y))

    return data
  end

gen_data (generic function with 1 method)

In [12]:
data = gen_data(n = 500000, pZ1 = 0.8);
first(data, 6)

,Obs_ID,Y,p_Y_given_Z_1,p_Y_given_Z_2
,Int64,Float64,Float64,Float64
1,1,3.33422,0.00153772,0.163816
2,2,0.507834,0.350678,0.131044
3,3,-0.849431,0.278119,0.00688392
4,4,-0.803331,0.288919,0.00784193
5,5,-1.98068,0.0561076,0.000144556
6,6,0.150476,0.394451,0.0721284


In [13]:
function fit_model!(
    data; 
    pi_hat_0 = 0.5, 
    tolerance = 0.0001,
    max_iterations = 1000,
    progress = DataFrame(
        iter = 0:max_iterations, 
        pi_hat = Vector{Float64}(undef, max_iterations+1), 
        ll = Vector{Float64}(undef, max_iterations+1), 
        ll_diff = Vector{Float64}(undef, max_iterations+1)
        ),
    verbose = false
    )

    # initial E step, to perform needed calculations for initial likelihood:
    E_step!(data, pi_hat_0) 
    ll = loglik(data)
    progress[1, :] = (0, pi_hat_0, ll, NaN)
    
    last_iter = 0
    for i in 1:max_iterations

        # M step: re-estimate parameters
        pi_hat = M_step(data)

        # E step: re-compute distribution of missing variables, using parameters
        E_step!(data, pi_hat)
        
        # Assess convergence

        ## save the previous log-likelihood so we can test for convergence
        ll_old = ll
        
        ## here's the new log-likelihood
        ll = loglik(data)

        ll_diff = ll - ll_old
        
        progress[i+1,:] = (i, pi_hat, ll, ll_diff)
        if(verbose)
            print(progress[i+1,:])
        end
        
        if ll_diff < tolerance
            last_iter = i
            break
        end
    end
    return progress[1:(last_iter + 1), :]
end

fit_model! (generic function with 1 method)

In [14]:
function E_step!(data, pi_hat)
    @transform!(data, :pY_Z1 = :p_Y_given_Z_1 .* pi_hat)
    @transform!(data, :pY_Z2 = :p_Y_given_Z_2 .* (1- pi_hat))
    @transform!(data, :pY = :pY_Z1 + :pY_Z2)
    @transform!(data, :pZ1_given_Y = :pY_Z1 ./ :pY)
end

E_step! (generic function with 1 method)

In [15]:
function M_step(data)
    data[:, :pZ1_given_Y] |> mean
end

M_step (generic function with 1 method)

In [16]:
function loglik(data)
    data[:, :pY] .|> log |> sum
end

loglik (generic function with 1 method)

In [17]:
@time progress = fit_model!(data, tolerance = .00001);


  0.567804 seconds (881.08 k allocations: 550.465 MiB, 40.88% gc time, 36.13% compilation time)


The first run requires compilation; on subsequent calls it will be faster:

In [18]:
@time progress = fit_model!(data, tolerance = .00001)

  0.156007 seconds (13.08 k allocations: 504.204 MiB, 18.83% gc time)


,iter,pi_hat,ll,ll_diff
,Int64,Float64,Float64,Float64
1,0,0.5,-8.77311e5,NaN
2,1,0.665296,-8.36758e5,40553.1
3,2,0.7376,-8.27786e5,8972.27
4,3,0.770088,-8.25704e5,2081.69
5,4,0.785471,-8.25196e5,508.161
6,5,0.793015,-8.25068e5,128.053
7,6,0.796789,-825035.0,32.8407
8,7,0.798697,-8.25027e5,8.50124
9,8,0.799667,-8.25024e5,2.21134
